In [27]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import pandas as pd

In [28]:
df = pd.read_csv('tfid_post_vectorize_agg.csv')

In [29]:
df.head()

,post_id,00,000,000th,0079,01,02,030,039,04,...,पह,নত,არაბული,თამთა,თემო,ოდიკაძე,ონისე,蔡小煒,___author,___bot_status
0,3brkjs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_____goats,0
1,32w130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_____goats,0
2,31c3d7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_____goats,0
3,3em4to,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,__8ball__,0
4,3coa1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,__8ball__,0


In [30]:
X = df.drop(['___author', '___bot_status', "post_id"], axis=1)
y = df['___bot_status']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

def pipe_maker(classifier):
    pipe = Pipeline([('clf', classifier)])
    return pipe

In [33]:
def gridsearch_maker(pipeline, params):
    grid = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='f1_weighted')
    return grid

In [34]:
def find_best_recall(gridsearch):
    gridsearch.fit(X_train, y_train)
    print('Best score:', gridsearch.best_score_)
    print('Best parameters:', gridsearch.best_params_)
    print('Best estimator:', gridsearch.best_estimator_)
    return gridsearch.best_estimator_

In [35]:
pipe_etclf = pipe_maker(RandomForestClassifier(),)
params_etclf = {'clf__n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                'clf__criterion': ['gini', 'entropy'],
                'clf__max_features': ['auto', 'sqrt', 'log2'],
                'clf__max_depth': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

In [36]:
# find_best_recall(gridsearch_maker(pipe_etclf, params_etclf))

In [37]:
clf = RandomForestClassifier(n_estimators=10, criterion='gini', max_features='auto', max_depth=None)

In [38]:
clf.fit(X_train, y_train)
print('Training set score: {:.3f}'.format(clf.score(X_train, y_train)))
print('Test set score: {:.3f}'.format(clf.score(X_test, y_test)))
print(classification_report(y_test, clf.predict(X_test)))
print(confusion_matrix(y_test, clf.predict(X_test)))

Training set score: 0.988
Test set score: 0.755
              precision    recall  f1-score   support

           0       0.72      0.80      0.76      1057
           1       0.80      0.72      0.75      1166

    accuracy                           0.75      2223
   macro avg       0.76      0.76      0.75      2223
weighted avg       0.76      0.75      0.75      2223

[[843 214]
 [331 835]]


In [39]:
predictions = clf.predict(X)

In [40]:
df['predictions'] = predictions
df_adjusted = df[['___author', '___bot_status', "post_id", 'predictions']]

In [42]:
df_adjusted.to_csv('predictions_rf.csv', index=False)

In [45]:
#export model with pickl
import pickle

In [46]:
#export clf
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [47]:
#get list of all feature names
feature_names = list(X.columns)

In [49]:
#save list of feature names to file
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)